# AWS Glue Studio Notebook

Analise atentamente os comandos a seguir e execute-os em sequência.

Caso você não tenha familiaridade com Jupyter notebooks, aqui está um mini tutorial para começar:

**Para executar células**: Um notebook é composto por células. Cada célula pode conter código Python ou texto formatado. Para executar o código em uma célula, clique na célula para selecioná-la, depois pressione `Shift+Enter`. O resultado do código será exibido abaixo da célula.

**Para adicionar novas células**: Você pode adicionar novas células clicando no botão "+" na barra de ferramentas na parte superior. A nova célula será adicionada abaixo da célula atualmente selecionada.

**Salvar o notebook**: Você pode salvar o notebook clicando no botão de **Save** na parte superior.

**Interromper o notebook**: Se quiser sair, não esqueça de interromper o notebook utilizando o botão **Stop notebook**. Isso garante que os recursos alocados para o notebook sejam liberados e você não seja cobrado por isso.

### 1. Iniciando uma sessão interativa.


In [ ]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
import boto3
import time
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  

print("Criando o contexto do Glue")
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

### 2. Definindo a variável de ambiente `s3_bucket`

In [ ]:
print("Definindo a variável s3_bucket que vamos utilizar ao longo do código")
s3_bucket = ""
s3_client = boto3.client('s3')
response = s3_client.list_buckets()

for bucket in response['Buckets']:
    if bucket['Name'].startswith('lab-data-eng-'):
        s3_bucket = bucket['Name']

print("O bucket que vamos utilizar é: " + s3_bucket)

### 3. Criando um DynamicFrame a partir de uma tabela do Glue Data Catalog


#### Clientes

In [ ]:
dyfClientes = glueContext.create_dynamic_frame.from_catalog(database='ecommerce', table_name='clientes')
dyfClientes.printSchema()

#### Pedidos

In [ ]:
dyfPedidos = glueContext.create_dynamic_frame.from_catalog(database='ecommerce', table_name='pedidos')
dyfPedidos.printSchema()

### 4. Convertendo o DynamicFrame para um Spark DataFrame e apresentando algumas linhas


#### Clientes

In [ ]:
dfCli = dyfClientes.toDF()
dfCli.show()

#### Pedidos

In [ ]:
dfPed = dyfPedidos.toDF()
dfPed.show()

### 5. Escrevendo os dados do DynamicFrame em um local do S3 e atualizando o Glue Data Catalog

#### Clientes

In [ ]:
s3output = glueContext.getSink(
  path="s3://" + s3_bucket + "/stage/ecommerce/clientes/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="ecommerce", catalogTableName="clientes_parquet"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(dyfClientes)

#### Pedidos

In [ ]:
s3output = glueContext.getSink(
  path="s3://" + s3_bucket + "/stage/ecommerce/pedidos/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="ecommerce", catalogTableName="pedidos_parquet"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(dyfPedidos)

### 6. Verificando os arquivos Parquet gerados

In [ ]:
session = boto3.Session()
s3 = session.resource('s3')
lab_eng_bucket = s3.Bucket(s3_bucket)

print("Arquivos de clientes em stage")
for clientes_object in lab_eng_bucket.objects.filter(Prefix="stage/ecommerce/clientes/"):
    print(clientes_object.key)

print("Arquivos de pedidos em stage")
for pedidos_object in lab_eng_bucket.objects.filter(Prefix="stage/ecommerce/pedidos/"):
    print(pedidos_object.key)

### 7. Verificando a criação da tabela `pedidos_parquet`

> ##### Atenção!
> Normalmente esse tipo de verificação visual não fica dentro do job. Neste laboratório só incluí por comodidade.

In [ ]:
!aws glue get-tables --database-name 'ecommerce'  --query "TableList[].Name"